In [1]:
import pandas
import nltk

from typing import List, Dict
from unidecode import unidecode
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from scipy import sparse

In [2]:
# loading dependencies
data = pandas.read_csv("estadao_noticias_eleicao.csv")

nltk.download('stopwords')

# cleaning null data
data = data.fillna('')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# joyning column of title and sub-title of artile with column of content.
data['articles'] = data['titulo']  + ' ' + data['subTitulo'] + ' ' + data['conteudo']

# lambda funcion to normalize text to lower case.
normalize = lambda text: unidecode(text.lower())

# lambda function to split text in tokens.
tokenize = lambda row: row.split()

# normalizing and tokenizing articles.
data['articles'] = data['articles'].apply(normalize)
data['tokens'] = data['articles'].apply(tokenize)

In [4]:
def create_index(matrix_of_tokens: List[str], doc_ids: List[int]) -> Dict[str, List[int]]:
    """Create a inverted index with all tokens and yours document Ids.
    :param matrix_of_tokens: matrix of article tokens lists.
    :param doc_ids: list of document ids of all articles.
    :returns: A inverted index with all tokens and yours document Ids.
    """
    index = {}
    for i in range(len(matrix_of_tokens)):
        for token in set(matrix_of_tokens[i]):
            if token in index.keys():
                index[token].append(doc_ids[i])
            else:
                index[token] = [doc_ids[i]]
    return index

In [5]:
# creating the inverted index
inverted_index = create_index(data['tokens'], data['idNoticia'])

In [6]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [7]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = data['tokens'].apply(lambda tokens: [token for token in tokens if token not in stopword_])
                                     
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [8]:
matrix, vocab = co_occurrence_matrix(tokens)

In [9]:
consultable_matrix = matrix.tocsr()

def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [10]:
tree = list(zip(matrix.row, matrix.col, matrix.data))

In [11]:
index_to_vocab = [k for k, v in vocab.items()]

In [12]:
def get_related_words(word: str) -> List[str]:
    """Returns the top three words with co occurrence with the passed word.
    :param word: passed word for get co occurrence.
    :returns: A array with top three words with co occurrence with the passed word.
    """
    index = index_to_vocab.index(word)
    filtered = list(filter((lambda x: x[0] == index or x[1] == index), tree))
    top_tree = list(map((lambda x: index_to_vocab[x[0]] if x[0] != index else index_to_vocab[x[1]]), filtered[:3]))
    return top_tree

In [13]:
def expand_search(query: str) -> List[str]:
    words = list(set(query.split(' ')))
    related_words = list(map(lambda w: get_related_words(w), words))
    related_words = [word for list_of_words in related_words for word in list_of_words]
    words += related_words
    return words

In [14]:
def search(query: str) -> List[int]:
    words = expand_search(query)
    ids = [inverted_index.get(word) for word in words]
    ids = [id for list_of_ids in ids for id in list_of_ids]
    return ids

Respondendo as Perguntas
----------------------------------------

Para isso foram escolhidas 3 palavras:
* Dilma
* Lula
* Temer

In [15]:
dilma = get_related_words('dilma')
lula = get_related_words('lula')
temer = get_related_words('temer')

print("Para dilma as 3 palavras relacionadas são:", dilma)
print("Para lula as 3 palavras relacionadas são:", lula)
print("Para temer as 3 palavras relacionadas são:", temer)

Para dilma as 3 palavras relacionadas são: ['cabras', 'arroubos', 'sri']
Para lula as 3 palavras relacionadas são: ['ultimos,', 'compartimentada', 'habilidade."']
Para temer as 3 palavras relacionadas são: ['arbitrariedade,', 'estocolmo,', '"exaustivamente']


        Os resultados parecem fazer bastante sentido, pois para os dois primeiros nomes
    os resultados são basicamente os sobrenomes dos mesmos, para os dois primeiros casos
    também se vê o problema de repetição te termos por não ter sido filtrado os termos 
    com vírgulas no final. Para a terceira palavra escolhida também uma das três palavras
    foi um dos nomes do mesmo, e outra das palavras é nome do partido a qual pertence 
    michel temer, apenas uma das palavras não dá pra se dizer com certeza se ela está 
    relacionada.
        No geral a expanssão das consultas parece estar funcionando bem, trazendo palavras 
    que parecem estar bastante relacionadas com as passadas.

        Na minha opinião a expanssão das consultas é mais adequada pra melhorar o recall,
    pois o recall é a fração dos documentos relevantes que são recuperados com êxito
    então quanto mais docs são recuperados, mais chances de ter documento relevantes.